# Bird Classifier
You see a beautiful bird and are wondering what kind of species it belongs to? This classifier will give you an quick answer (capable of determining 400 bird species). Simply take a picture of the bird and click "upload" to find out!

**Note**:This classifier is only able to produce one bird species at a time, so it's not a good idea to include multiple species or something other than a bird in a picture.

In [3]:
# !pip install -q git+https://github.com/fastai/fastai
# !pip install -q timm

In [2]:
import requests
import base64
import json
from fastai.vision.all import *
from fastai.vision.widgets import *

In [3]:
# learn = load_learner('first_efficientnet_b3a.pkl')
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_upload = widgets.FileUpload()

In [4]:
api = "12f47e5d16c17cb91aa8d9be0c9ca121"
imgbburl = "https://api.imgbb.com/1/upload"

In [5]:
def data_change(change):
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    lbl_pred.value = "Processing"
    payload = {
    'expiration':60,
    'key': api,
    'image': base64.b64encode(btn_upload.data[-1])
    }
    respond = requests.post(url,data=payload)
    if respond.status_code == 200:
        img_url = json.loads(respond.text)['data']['image']['url']
        modelurl = "https://birdclassifier.azurewebsites.net/api/classify?code=pUSRaZ83c390WHyXFdD94X1EiAeFC58kP6loTb9cGca52/q3Eipa/Q==&img="
        model_res = requests.get(modelurl + img_url)
        prediction = json.loads(model_res.text)
        lbl_pred.value = f'Prediction: {prediction["prediction"]}; Probability: {prediction["confidence"]}'
    else:
        lbl_pred.value = "Failed to upload image"

btn_upload.observe(data_change, names=['data'])

In [6]:
VBox([widgets.Label('Please upload your bird picture'),
      btn_upload, out_pl, lbl_pred])